In [1]:
import sys
import os.path
import datetime
import subprocess
import dateutil.parser
import datetime
from datetime import timedelta
from datetime import date
from pyspark import *
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark import sql
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyarrow import csv
import feather

spark = SparkSession \
    .builder \
    .appName("Foo") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "12g") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)





In [2]:
spark.sparkContext._conf.getAll()

[('spark.driver.memory', '12g'),
 ('spark.driver.maxResultSize', '8g'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Foo'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '42400'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '192.168.1.13'),
 ('spark.app.id', 'local-1572309868928'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
%time df = spark.read.csv("/home/aaron/Downloads/20171223_v1_productlistings_0000*.csv.gz", inferSchema=True, header=True)

In [3]:
df2 = df.select('chain','banner','channel_method','postal_code','sku')

In [5]:
df2.persist(StorageLevel.DISK_ONLY)

DataFrame[chain: string, banner: string, channel_method: string, postal_code: int, sku: bigint]

In [7]:
df2.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45303)
Traceback (most recent call last):
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:45303)

In [ ]:
df.coalesce(1).write.csv("/home/aaron/Downloads/onecsv/onething.csv")

In [ ]:
mport feather
path = 'my_data.feather'
feather.write_dataframe(df, path)

In [6]:
%time pdf = df2.toPandas()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 2182, in _collectAsArrow
    return list(_load_from_socket((port, auth_secret), ArrowStreamSerializer()))
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/pyspark/serializers.py", line 208, in load_stream
    for batch in reader:
  File "pyarrow/ipc.pxi", line 300, in __iter__
  File "pyarrow/ipc.pxi", line 316, in pyarrow.lib._CRecordBatchReader.read_next_batch
  File "pyarrow/error.pxi", line 80, in pyarrow.lib.check_status
pyarrow.lib.ArrowIOError: Expected to be able to read 443984 bytes for message body, got 245233

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
p

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:45303)

In [9]:
pdf

,chain,banner,channel_method,postal_code,sku
0,Walmart,Walmart,instore,50702,51348482
1,Walmart,Walmart,pickup,75023,3100532056
2,Walmart,Walmart,pickup,8619,3000182824
3,Walmart,Walmart,pickup,77095,3000682242
4,Walmart,Walmart,instore,34114,10321378
...,...,...,...,...,...
17991509,Walmart,Walmart,instore,81212,10535943
17991510,Walmart,Walmart,instore,89015,10292305
17991511,Walmart,Walmart,pickup,85225,1015303
17991512,Walmart,Walmart,pickup,80214,3000885460


In [ ]:
import pandas as pd
import glob

path = r'/home/aaron/Downloads/' # use your path
all_files = glob.glob(path + "/*.csv.gz")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)